# Environmental Data Analysis 

In [1]:
import ee
import geemap

In [2]:
# Initialize Earth Engine
ee.Authenticate()
ee.Initialize(project="earthengine-first-project")

In [3]:
# Initialize map
m = geemap.Map()

# Load study area shapefile
fc = geemap.shp_to_ee(r"E:\New folder (3)\FFRM_Lumbini\Study area\Lumbini_province.shp")
style = {"color": "black", "fillColor": "#00000000", "width": 2}
m.add_layer(fc.style(**style), {}, "Lumbini Province")
m.centerObject(fc)

## Elevation (SRTM 30)

In [4]:
# Load and process SRTM elevation data
srtm = ee.Image('USGS/SRTMGL1_003')
elevation = srtm.clip(fc)

# Visualization parameters
elevation_vis = {
    'min': 0,
    'max': 4000,
    'palette': ['0000ff', '00ffff', '00ff00', 'ffff00', 'ff0000']
}

m.add_layer(elevation, elevation_vis, "SRTM Elevation")

# Export elevation data
elevation_export = elevation.unmask()
geemap.ee_export_image_to_drive(
    elevation_export, 
    description="lumbini_elevation.tif",
    scale=30,
    region=fc.geometry().bounds(),
)

## Land Surface Temperature (MODIS)

In [5]:
# Load and process MODIS LST data
lst_collection = (ee.ImageCollection('MODIS/061/MOD11A1')
                  .filterDate('2000-01-01', '2025-08-30')
                  .select('LST_Day_1km')
                  .map(lambda img: img.clip(fc)))

# Convert to Celsius
def to_celsius(img):
    return img.multiply(0.02).subtract(273.15).copyProperties(img, img.propertyNames())

lst_celsius = lst_collection.map(to_celsius)
lst_mean = lst_celsius.mean()

# Visualization parameters
lst_vis = {
    "min": 10,
    "max": 40,
    "palette": ['040274','0502b8','0aab1e','e7eb05','ff4a2d','e90000']
}

m.add_layer(lst_mean, lst_vis, "Mean LST (°C)")

# Export LST data
lst_export = lst_mean.unmask()
geemap.ee_export_image_to_drive(
    lst_export,
    description="lumbini_lst_mean.tif",
    scale=1000,
    region=fc.geometry().bounds(),
)

## Precipitation (CHIRPS)

In [6]:
# Load and process CHIRPS precipitation data
chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY").select("precipitation")

# Function for yearly total precipitation
def yearly_total_precip(year):
    start = ee.Date.fromYMD(year, 1, 1)
    end = start.advance(1, "year")
    yearly = chirps.filterDate(start, end)
    return yearly.sum().clip(fc).set("year", year)

# Generate yearly totals and compute average
years = list(range(2000, 2026))
yearly_totals = ee.ImageCollection([yearly_total_precip(y) for y in years])
avg_yearly_total = yearly_totals.mean().rename("Avg_Yearly_Total_Precip")

# Visualization parameters
precip_vis = {
    "min": 500,
    "max": 2500,
    "palette": ["white", "blue", "green", "yellow", "red"]
}

m.add_layer(avg_yearly_total, precip_vis, "Avg Yearly Total Precip (2000-2025)")

# Export precipitation data
precip_export = avg_yearly_total.unmask()
geemap.ee_export_image_to_drive(
    precip_export,
    description="lumbini_precipitation.tif",
    scale=5500,
    region=fc.geometry().bounds(),
)

## NDVI (Landsat 8/9)

In [7]:
# Load and process Landsat data for NDVI calculation
landsat = (
    ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
    .filterBounds(fc)
    .filterDate("2024-01-01", "2024-12-31")
    .filter(ee.Filter.lt("CLOUD_COVER", 20))
)

def calc_ndvi(image):
    # Apply scaling factors
    red = image.select("SR_B4").multiply(0.0000275).add(-0.2)
    nir = image.select("SR_B5").multiply(0.0000275).add(-0.2)
    
    # Calculate NDVI
    ndvi = nir.subtract(red).divide(nir.add(red)).rename("NDVI")
    
    # Mask invalid pixels
    mask = red.gte(0).And(red.lte(1)).And(nir.gte(0)).And(nir.lte(1))
    ndvi = ndvi.updateMask(mask)
    
    return image.addBands(ndvi)

# Apply NDVI function and compute median
ndvi_collection = landsat.map(calc_ndvi)
ndvi_2024 = ndvi_collection.select("NDVI").median().clip(fc)

# Visualization parameters
ndvi_vis = {"min": -1, "max": 1, "palette": ["brown", "yellow", "green"]}

m.add_layer(ndvi_2024, ndvi_vis, "Median NDVI 2024")

# Export NDVI data
ndvi_export = ndvi_2024.unmask()
geemap.ee_export_image_to_drive(
    ndvi_export,
    description="lumbini_ndvi_2024.tif",
    scale=30,
    region=fc.geometry().bounds(),
)

## Display Final Map

In [8]:
# Add layer control and display the map
m.add_layer_control()
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…